In [1]:
import cv2
from fpdf import FPDF
import cv2.aruco as aruco
import numpy as np
from glob import glob
from matplotlib import pyplot as plt
import pandas as pd

In [2]:
# Define the parameters for the Charuco board
num_squares_x = 5
num_squares_y = 7
square_length = 100
marker_length = 80

# Create the Charuco board
board = aruco.CharucoBoard_create(
    squaresX=num_squares_x,
    squaresY=num_squares_y,
    squareLength=square_length,
    markerLength=marker_length,
    dictionary=aruco.Dictionary_get(aruco.DICT_4X4_50)
)

In [3]:
camera_matrix = np.loadtxt("camera_matrix.csv", delimiter=",")
dist_coeffs = np.loadtxt("distortion_coefficients.csv",
                         delimiter=",")

In [4]:
# Load the calibration images
calibration_images = [glob(i + "/*.jpg")
                      for i in glob("../../bilderserien/charuco/*")]

In [5]:
liste = []

In [6]:
for i, serie in enumerate(calibration_images):
    for pfad in serie:
        image = cv2.imread(pfad)
        gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
        corners, ids, rejectedImgPoints = aruco.detectMarkers(
            gray, board.dictionary)
        aruco.refineDetectedMarkers(
            gray, board, corners, ids, rejectedImgPoints)
        # gray = cv2.cvtColor(gray, cv2.COLOR_BGR2RGB)
        if corners:  # if there is at least one marker detected
            charucoretval, charucoCorners, charucoIds = aruco.interpolateCornersCharuco(
                corners, ids, gray, board)
            im_with_charuco_board = aruco.drawDetectedCornersCharuco(
                gray, charucoCorners, charucoIds, (0, 255, 0))
            rvec, tvec = None, None
            retval, rvec, tvec = aruco.estimatePoseCharucoBoard(
                charucoCorners, charucoIds, board, camera_matrix, dist_coeffs, rvec, tvec)  # posture estimation from a charuco board

            if retval == True:
                # axis length 100 can be changed according to your requirement
                # im_with_charuco_board = cv2.drawFrameAxis(
                #   im_with_charuco_board, camera_matrix, dist_coeffs, rvec, tvec, 100)
                gray = aruco.drawDetectedCornersCharuco(
                    gray, charucoCorners, charucoIds)
                gray = cv2.drawFrameAxes(
                    gray, camera_matrix, dist_coeffs, rvec, tvec, 100, 10)
                liste.append([i, int(pfad[-6: -4]), tvec[0][0], tvec[1][0],  tvec[
                    2][0], rvec[0][0], rvec[1][0],  rvec[2][0]])

            # plt.figure(figsize=(14, 11))
            # plt.imshow(gray)
            # plt.title(i)
            # plt.show()

In [7]:
df = pd.DataFrame(liste,
                  columns=["serie", "image", "t1", "t2", "t3", "r1", "r2", "r3"])
df

,serie,image,t1,t2,t3,r1,r2,r3
0,0,1,-550.974101,-81.331424,942.317202,1.559219,0.264291,-1.320321
1,2,13,-955.788834,451.080072,1310.147740,0.505830,-1.377375,-0.603576
2,2,16,-812.670406,169.877476,1880.000482,0.479745,-0.634539,-0.219412
3,2,17,-822.765808,-251.709110,1440.611030,0.160580,-0.682056,-0.107797
4,2,18,-838.470303,-591.872096,1407.284071,-0.036649,-0.720462,-0.004840
...,...,...,...,...,...,...,...,...
75,12,16,604.040996,559.839317,1396.544309,0.345235,0.678844,2.267590
76,12,17,599.970712,-33.608968,895.810614,0.154415,1.027134,2.104221
77,12,19,356.172381,612.257407,1071.375835,1.010535,1.089829,2.596355
78,12,20,370.224820,199.780075,593.908696,0.933911,1.430183,2.291295


In [14]:
df.groupby("image").count()

,serie,t1,t2,t3,r1,r2,r3
image,,,,,,,
1,5,5,5,5,5,5,5
2,2,2,2,2,2,2,2
3,1,1,1,1,1,1,1
4,7,7,7,7,7,7,7
5,2,2,2,2,2,2,2
6,2,2,2,2,2,2,2
7,8,8,8,8,8,8,8
8,4,4,4,4,4,4,4
10,6,6,6,6,6,6,6


In [67]:
img7 = df.where((df["image"] == 7)).dropna()
img4 = df.where((df["image"] == 4)).dropna()
img7

,serie,image,t1,t2,t3,r1,r2,r3
15,4.0,7.0,36.011018,-656.787022,1165.368269,0.038135,-1.233551,1.085748
19,5.0,7.0,-415.002027,656.655016,2067.548523,-0.781761,-0.272662,-0.641544
32,6.0,7.0,490.065975,-418.025987,2081.267189,0.403238,0.683502,0.428722
36,7.0,7.0,-764.199879,-177.910392,1818.164990,0.573171,-0.733257,-0.381310
45,8.0,7.0,-171.212151,-474.484930,1500.886115,-0.769510,0.230422,0.352991
52,9.0,7.0,-515.079963,-508.449381,815.074447,0.676643,-1.047424,-0.448884
62,11.0,7.0,-95.795913,-202.254298,1792.420644,-0.320769,0.750961,0.147685
69,12.0,7.0,-320.382987,-118.311466,1808.554609,-0.866787,-0.578679,0.445118


In [72]:
v = img4.set_index('serie').join(
    img7.set_index('serie'), lsuffix="_a", rsuffix="_b").dropna()

# v["r1"] = (v["r1_a"] - v["r1_b"])
# v["r2"] = (v["r2_a"] - v["r2_b"])
# v["r3"] = (v["r3_a"] - v["r3_b"])
v

,image_a,t1_a,t2_a,t3_a,r1_a,r2_a,r3_a,image_b,t1_b,t2_b,t3_b,r1_b,r2_b,r3_b
serie,,,,,,,,,,,,,,
6.0,4.0,-306.394308,-292.721764,2097.910395,0.314681,-0.012074,0.005843,7.0,490.065975,-418.025987,2081.267189,0.403238,0.683502,0.428722
7.0,4.0,-922.584890,395.419633,1184.977896,0.392191,-1.479335,-0.653040,7.0,-764.199879,-177.910392,1818.164990,0.573171,-0.733257,-0.381310
8.0,4.0,-494.731511,-197.242541,1258.088748,-0.851860,-0.133291,-0.395791,7.0,-171.212151,-474.484930,1500.886115,-0.769510,0.230422,0.352991
9.0,4.0,-391.480617,-247.992338,499.610345,0.426218,-1.852030,-0.663809,7.0,-515.079963,-508.449381,815.074447,0.676643,-1.047424,-0.448884
11.0,4.0,-456.827826,79.272034,1544.546773,-0.233959,0.220397,-0.469495,7.0,-95.795913,-202.254298,1792.420644,-0.320769,0.750961,0.147685
12.0,4.0,-577.274753,249.734759,1429.180717,-1.226081,-0.855648,-0.285769,7.0,-320.382987,-118.311466,1808.554609,-0.866787,-0.578679,0.445118


In [82]:
for w in v[['r1_a', 'r2_a', 'r3_a', 'r1_b', 'r2_b', 'r3_b']].values:
    a = cv2.Rodrigues(w[:3])[0]
    b = cv2.Rodrigues(w[3:])[0]
    k = a@np.ones(3)
    print(k/k[2])

[0.77082068 0.50680577 1.        ]
[-0.58073968  0.20184313  1.        ]
[6.13342208 4.79958101 1.        ]
[-1.17679598  0.41190443  1.        ]
[2.96738671 1.08615324 1.        ]
[10.40031243 17.97164491  1.        ]


In [63]:
import plotly.graph_objs as go

coords_img = []
for _, p in df.where(df["serie"] == 12).dropna().iterrows():
    rt = p[['r1', 'r2', 'r3']].to_numpy(dtype=np.float64)
    R = cv2.Rodrigues(rt)[0]
    R = np.linalg.inv(R)
    t = p[['t1', 't2', 't3']].to_numpy(dtype=np.float64).T
    t = -R@t
    coords_img.append(t)
coords_img = np.array(coords_img)

# Create a 3D scatter plot
fig = go.Figure(data=[go.Scatter3d(x=coords_img[:, 0], y=coords_img[:, 1], z=coords_img[:, 2],
                                   mode='markers', name='Kameras')])

# Add labels to the plot
fig.update_layout(scene=dict(xaxis_title='X', yaxis_title='Y', zaxis_title='Z'),
                  title='Rotation and Translation Vectors in 3D')
fig.layout.scene.camera.projection.type = "orthographic"
fig.show()